In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
from scipy.interpolate import interp1d
import gc
import soundfile as sf
# Librosa Libraries
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score, label_ranking_average_precision_score
from sklearn.model_selection import GroupKFold
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append('../input/iterative/iterative_stratification-0.1.6-py3-none-any.whl')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
trainfiles = glob.glob( '../input/rfcx-species-audio-detection/train/*.flac' )
testfiles = glob.glob( '../input/rfcx-species-audio-detection/test/*.flac' )
len(trainfiles), len(testfiles), trainfiles[0]

In [ ]:
traint = pd.read_csv( '../input/rfcx-species-audio-detection/train_tp.csv' )
traint['t_dif'] = traint['t_max'] - traint['t_min']
traint['f_dif'] = traint['f_max'] - traint['f_min']

trainf = pd.read_csv( '../input/rfcx-species-audio-detection/train_fp.csv' )
trainf['t_dif'] = trainf['t_max'] - trainf['t_min']
trainf['f_dif'] = trainf['f_max'] - trainf['f_min']

traint.shape, trainf.shape

In [ ]:
traint.head()

In [ ]:
trainf.head()

In [ ]:
trainf.t_min.hist(bins=100)
trainf.t_max.hist(bins=100, alpha=0.5)

In [ ]:
trainf.f_min.hist(bins=20)
trainf.f_max.hist(bins=20, alpha=0.5)

In [ ]:
trainf.t_dif.hist(bins=100)


In [ ]:
trainf.f_dif.unique()

In [ ]:
trainf.nunique()

In [ ]:
traint.nunique()

In [ ]:
data, samplerate = sf.read(trainfiles[0]) 
print( data.shape, samplerate )
librosa.display.waveplot(y = data, sr = samplerate, color = "#B14D")

In [ ]:
traint.describe()

In [ ]:
trainf.describe()

# Min and Max frequencies are: 93 and 10687

In [ ]:
"""
TRAIN = []
TARGET = []
for i in tqdm(range(traint.shape[0])):

    fn = traint.recording_id.values[i]
    tmin = traint.t_min.values[i]
    tmax = traint.t_max.values[i]
    fmin = traint.f_min.values[i]
    fmax = traint.f_max.values[i]
    #print(tmin,tmax, fmin,fmax )

    data, samplerate = sf.read( '../input/rfcx-species-audio-detection/train/'+fn+'.flac')
    #print( data.shape, samplerate )
    var_time = np.arange(0,data.shape[0]) / samplerate

    data = data[ np.where( (var_time>=tmin)&(var_time<=tmax) )[0] ]

    varfft = np.abs( np.fft.fft(data)[:(len(data)//2)] )
    x = np.linspace(0, len(varfft), num=len(varfft), endpoint=True)
    f1 = interp1d(x, varfft, kind='cubic')
    x = np.linspace(0, len(varfft), num=1000, endpoint=True)
    varfft = f1(x)
    
    TRAIN.append( varfft )
    TARGET.append( traint.species_id.values[i] )
    
FT = np.stack(TRAIN)
TARGET = np.array(TARGET)
FT.shape, len(TARGET)
"""
data = np.load('../input/training-testing-data/FT.npz')
FT = data['a']
data = np.load('../input/training-testing-data/TARGET.npz')
TARGET = data['a']

In [ ]:
np.unique(TARGET, return_counts=True)

In [ ]:
"""
from joblib import Parallel, delayed

def extract_features( fn ):
    data, samplerate = sf.read( '../input/rfcx-species-audio-detection/train/'+fn+'.flac')

    varfft = np.abs( np.fft.fft(data)[:(len(data)//2)] )
    x = np.linspace(0, len(varfft), num=len(varfft), endpoint=True)
    f1 = interp1d(x, varfft, kind='cubic')
    x = np.linspace(0, len(varfft), num=1000, endpoint=True)
    varfft = f1(x)
    
    return varfft
    
FP = Parallel(n_jobs=4)(delayed(extract_features)(fn) for i in tqdm(trainf.recording_id.values))
FP = np.stack(FP)
gc.collect()
FP.shape
"""
data = np.load('../input/training-testing-data/FP.npz')
FP = data['a']

In [ ]:
"""
def extract_features( fn ):
    data, samplerate = sf.read(fn)

    varfft = np.abs( np.fft.fft(data)[:(len(data)//2)] )
    x = np.linspace(0, len(varfft), num=len(varfft), endpoint=True)
    f1 = interp1d(x, varfft, kind='cubic')
    x = np.linspace(0, len(varfft), num=1000, endpoint=True)
    varfft = f1(x)
    
    return varfft
    
TEST = Parallel(n_jobs=4)(delayed(extract_features)(fn) for fn in tqdm(testfiles))
TEST = np.stack(TEST)
gc.collect()
TEST.shape

"""
data = np.load('../input/training-testing-data/TEST.npz')
TEST = data['a']

In [ ]:
TRAIN = np.vstack( (FT, FP) )
TRAIN.shape

In [ ]:
tt = traint[['recording_id','species_id']].copy()
tf = trainf[['recording_id','species_id']].copy()

tf['species_id'] = -1

TRAIN_TAB = pd.concat( (tt, tf) )

for i in range(24):
    TRAIN_TAB['s'+str(i)] = 0
    TRAIN_TAB.loc[TRAIN_TAB.species_id==i,'s'+str(i)] = 1

TRAIN_TAB.shape

In [ ]:
TRAIN_TAB.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit( np.vstack((TRAIN,TEST)) )

TRAIN = std.transform(TRAIN)
TEST  = std.transform(TEST)
gc.collect()

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from tensorflow.keras import layers,regularizers,Sequential,backend,callbacks,optimizers,metrics,losses

In [ ]:
def create_model(num_columns,target_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5), 
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(500, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),        
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(target_columns, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss=losses.BinaryCrossentropy(label_smoothing=0.000001),metrics=['AUC']
                  )
    return model

In [ ]:
tar_col = []
for tgt in range(24):
    tar_col.append('s'+str(tgt))

In [ ]:
TRAIN_TARGETS = TRAIN_TAB[tar_col].copy()

In [ ]:
TRAIN = np.array(TRAIN,dtype='float64')
TRAIN_TARGETS = TRAIN_TARGETS[tar_col].values
TRAIN_TARGETS = np.array(TRAIN_TARGETS,dtype='float64')

In [ ]:
TRAIN.argmax()

In [ ]:
sub = pd.DataFrame({'recording_id': [f.split('/')[-1].split('.')[0] for f in testfiles] })
ytrain = np.zeros((TRAIN.shape[0],len(tar_col)))
ytest = np.zeros((TEST.shape[0],len(tar_col)))
N_SPLITS = 5 
SEED = 2
groups = TRAIN_TAB['recording_id'].values
for tar in range(24):
    print(f'---------------------TARGET{tar}------------------------')     
    for n, (tr, te) in enumerate(GroupKFold(N_SPLITS).split(TRAIN, TRAIN_TAB[tar_col[tar]],groups)):
                model = create_model(TRAIN.shape[1],1)
                print(f'FOLD-{n}')
                checkpoint_path_model = f'-model-repeat:{0}_Fold:{n}.hdf5'
                reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, epsilon=1e-4, mode='min')
                cb_checkpt = ModelCheckpoint(checkpoint_path_model, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
           
                model.fit(TRAIN[tr,:],
                      TRAIN_TARGETS[tr,tar],
                      validation_data=(TRAIN[te,:], TRAIN_TARGETS[te,tar]),
                      epochs= 30 , batch_size=128,
                      callbacks=[reduce_lr_loss, cb_checkpt], verbose=0,
                      )     
                os.remove(checkpoint_path_model)
                ytrain[te,tar] += model.predict(TRAIN[te,:])[:,0]
                ytest[:,tar] +=  model.predict(TEST)[:,0]/(N_SPLITS)
            
                FOLD_SCORE = roc_auc_score(TRAIN_TARGETS[te,tar],ytrain[te,tar])
                print(f'FOLD-{n}-AUC score = {FOLD_SCORE}')

In [ ]:
AUC_scores = []
for n  in range(len(tar_col)): 
    AUC_scores.append(roc_auc_score(TRAIN_TARGETS[:,n], ytrain[:,n]))
    print( f'Target{n} AUC', roc_auc_score(TRAIN_TARGETS[:,n], ytrain[:,n]) )
print(f'mean of AUC scores = {np.mean(AUC_scores)}') 
print(f'label_ranking_average_precision_score = {label_ranking_average_precision_score(TRAIN_TARGETS,ytrain)}')

In [ ]:
TRAIN_TAB[tar_col] =  ytrain
sub[tar_col] = ytest

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)